In [32]:
import os
import sys
import pandas as pd
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession, SQLContext

from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql.functions import udf, col

from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml import Pipeline


#temporary fix for running pyspark in jupyter notebook
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [33]:
# initialize spark session
spark = SparkSession.builder.master("local[2]").appName("Victim-Records").getOrCreate()

sc = spark.sparkContext

sqlContext = SQLContext(sc)


/Users/seantan88/anaconda3/lib/python3.11/site-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [34]:
# define the path to the data

VictimRecords = ['20160924_VictimRecords.txt','20170112_VictimRecords.txt','20180925_VictimRecords.txt', '20201024_VictimRecords.txt']
# define the schema
schema = StructType([
    StructField('CASE_ID', StringType(), True),
    StructField('PARTY_NUMBER', IntegerType(), True),
    StructField('VICTIM_ROLE', StringType(), True),
    StructField('VICTIM_SEX', StringType(), True),
    StructField('VICTIM_AGE', IntegerType(), True),
    StructField('VICTIM_DEGREE_OF_INJURY', StringType(), True),
    StructField('VICTIM_SEATING_POSITION', StringType(), True),
    StructField('VICTIM_SAFETY_EQUIP1', StringType(), True),
    StructField('VICTIM_SAFETY_EQUIP2', StringType(), True),
    StructField('VICTIM_EJECTED', StringType(), True),
])


# load the data, skip header
victim_df = spark.read.csv(path = VictimRecords, schema = schema).cache()
header = victim_df.first()
victim_df = victim_df.filter(col("CASE_ID") != header["CASE_ID"])
victim_df.take(5)

#### Debug: Reduce dataset for testing ####
#victim_df = victim_df.limit(10)




24/12/16 15:03:07 WARN CacheManager: Asked to cache already cached data.


[Row(CASE_ID=' 097293', PARTY_NUMBER=1, VICTIM_ROLE='2', VICTIM_SEX='M', VICTIM_AGE=20, VICTIM_DEGREE_OF_INJURY='0', VICTIM_SEATING_POSITION='3', VICTIM_SAFETY_EQUIP1='G', VICTIM_SAFETY_EQUIP2='-', VICTIM_EJECTED='0'),
 Row(CASE_ID=' 965874', PARTY_NUMBER=2, VICTIM_ROLE='2', VICTIM_SEX='M', VICTIM_AGE=19, VICTIM_DEGREE_OF_INJURY='0', VICTIM_SEATING_POSITION='3', VICTIM_SAFETY_EQUIP1='M', VICTIM_SAFETY_EQUIP2='G', VICTIM_EJECTED='0'),
 Row(CASE_ID='0000003', PARTY_NUMBER=1, VICTIM_ROLE='2', VICTIM_SEX='M', VICTIM_AGE=21, VICTIM_DEGREE_OF_INJURY='0', VICTIM_SEATING_POSITION='3', VICTIM_SAFETY_EQUIP1='G', VICTIM_SAFETY_EQUIP2='-', VICTIM_EJECTED='0'),
 Row(CASE_ID='0000005', PARTY_NUMBER=1, VICTIM_ROLE='1', VICTIM_SEX='M', VICTIM_AGE=44, VICTIM_DEGREE_OF_INJURY='2', VICTIM_SEATING_POSITION='1', VICTIM_SAFETY_EQUIP1='G', VICTIM_SAFETY_EQUIP2='-', VICTIM_EJECTED='0'),
 Row(CASE_ID='0000008', PARTY_NUMBER=1, VICTIM_ROLE='2', VICTIM_SEX='F', VICTIM_AGE=59, VICTIM_DEGREE_OF_INJURY='0', VICTIM_

In [35]:
victim_df.columns


['CASE_ID',
 'PARTY_NUMBER',
 'VICTIM_ROLE',
 'VICTIM_SEX',
 'VICTIM_AGE',
 'VICTIM_DEGREE_OF_INJURY',
 'VICTIM_SEATING_POSITION',
 'VICTIM_SAFETY_EQUIP1',
 'VICTIM_SAFETY_EQUIP2',
 'VICTIM_EJECTED']

In [36]:
victim_df.printSchema()

root
 |-- CASE_ID: string (nullable = true)
 |-- PARTY_NUMBER: integer (nullable = true)
 |-- VICTIM_ROLE: string (nullable = true)
 |-- VICTIM_SEX: string (nullable = true)
 |-- VICTIM_AGE: integer (nullable = true)
 |-- VICTIM_DEGREE_OF_INJURY: string (nullable = true)
 |-- VICTIM_SEATING_POSITION: string (nullable = true)
 |-- VICTIM_SAFETY_EQUIP1: string (nullable = true)
 |-- VICTIM_SAFETY_EQUIP2: string (nullable = true)
 |-- VICTIM_EJECTED: string (nullable = true)



In [37]:
victim_df.describe().show()

24/12/16 15:04:37 WARN MemoryStore: Not enough space to cache rdd_3_6 in memory! (computed 25.4 MiB so far)
24/12/16 15:04:39 WARN MemoryStore: Not enough space to cache rdd_3_7 in memory! (computed 7.5 MiB so far)
24/12/16 15:05:04 WARN MemoryStore: Not enough space to cache rdd_3_8 in memory! (computed 22.3 MiB so far)
24/12/16 15:05:05 WARN MemoryStore: Not enough space to cache rdd_3_9 in memory! (computed 3.9 MiB so far)
24/12/16 15:05:25 WARN MemoryStore: Not enough space to cache rdd_3_10 in memory! (computed 25.4 MiB so far)


+-------+--------------------+------------------+------------------+------------------+------------------+-----------------------+-----------------------+--------------------+--------------------+------------------+
|summary|             CASE_ID|      PARTY_NUMBER|       VICTIM_ROLE|        VICTIM_SEX|        VICTIM_AGE|VICTIM_DEGREE_OF_INJURY|VICTIM_SEATING_POSITION|VICTIM_SAFETY_EQUIP1|VICTIM_SAFETY_EQUIP2|    VICTIM_EJECTED|
+-------+--------------------+------------------+------------------+------------------+------------------+-----------------------+-----------------------+--------------------+--------------------+------------------+
|  count|            30396065|          30396065|          30396065|          30396065|          30396065|               30396065|               30396065|            30396065|            28568078|          30396065|
|   mean|3.570956518533931...|1.6305697464457982|1.8703510818017473|3.5555555555555554| 63.75028152492765|     1.9722526757425833|      

In [38]:
victim_df.filter(victim_df.VICTIM_AGE == 0).count()

24/12/16 15:05:43 WARN MemoryStore: Not enough space to cache rdd_3_6 in memory! (computed 14.7 MiB so far)
24/12/16 15:05:43 WARN MemoryStore: Not enough space to cache rdd_3_7 in memory! (computed 14.7 MiB so far)
24/12/16 15:05:43 WARN MemoryStore: Not enough space to cache rdd_3_9 in memory! (computed 3.9 MiB so far)
24/12/16 15:05:43 WARN MemoryStore: Not enough space to cache rdd_3_8 in memory! (computed 22.3 MiB so far)
24/12/16 15:05:44 WARN MemoryStore: Not enough space to cache rdd_3_10 in memory! (computed 25.4 MiB so far)


217309

In [39]:

# filter out unknown ages (998)
victim_df = victim_df.filter(victim_df.VICTIM_AGE != 998)

# change all 999 age instances to 0
victim_df = victim_df.withColumn('VICTIM_AGE', F.when(victim_df.VICTIM_AGE == 999, 0).otherwise(victim_df.VICTIM_AGE))


victim_df.describe().show()








24/12/16 15:07:16 WARN MemoryStore: Not enough space to cache rdd_3_6 in memory! (computed 25.4 MiB so far)
24/12/16 15:07:19 WARN MemoryStore: Not enough space to cache rdd_3_7 in memory! (computed 7.5 MiB so far)
24/12/16 15:07:57 WARN MemoryStore: Not enough space to cache rdd_3_8 in memory! (computed 22.3 MiB so far)
24/12/16 15:07:58 WARN MemoryStore: Not enough space to cache rdd_3_9 in memory! (computed 3.9 MiB so far)
24/12/16 15:08:23 WARN MemoryStore: Not enough space to cache rdd_3_10 in memory! (computed 25.4 MiB so far)


+-------+--------------------+------------------+------------------+------------------+------------------+-----------------------+-----------------------+--------------------+--------------------+-------------------+
|summary|             CASE_ID|      PARTY_NUMBER|       VICTIM_ROLE|        VICTIM_SEX|        VICTIM_AGE|VICTIM_DEGREE_OF_INJURY|VICTIM_SEATING_POSITION|VICTIM_SAFETY_EQUIP1|VICTIM_SAFETY_EQUIP2|     VICTIM_EJECTED|
+-------+--------------------+------------------+------------------+------------------+------------------+-----------------------+-----------------------+--------------------+--------------------+-------------------+
|  count|            29360479|          29360479|          29360479|          29360479|          29360479|               29360479|               29360479|            29360479|            27573502|           29360479|
|   mean|3.624734272311125...|1.6304027260590674|1.8611631661828245|3.5619047619047617|30.780437471745607|      2.009398080677406|  

In [40]:
victim_df.filter(victim_df.VICTIM_AGE == 0).count()

24/12/16 15:08:42 WARN MemoryStore: Not enough space to cache rdd_3_6 in memory! (computed 14.7 MiB so far)
24/12/16 15:08:42 WARN MemoryStore: Not enough space to cache rdd_3_7 in memory! (computed 14.7 MiB so far)
24/12/16 15:08:42 WARN MemoryStore: Not enough space to cache rdd_3_8 in memory! (computed 14.9 MiB so far)
24/12/16 15:08:42 WARN MemoryStore: Not enough space to cache rdd_3_9 in memory! (computed 14.7 MiB so far)
24/12/16 15:08:42 WARN MemoryStore: Not enough space to cache rdd_3_10 in memory! (computed 25.4 MiB so far)


217824

In [41]:
# count number of rows in victim_df
print("Number of rows in victim_df: "+ str(victim_df.count()))

# filter out all rows with null values
filtered_df = victim_df.dropna()

# count number of rows in filtered_df
print("Number of rows in filtered_df: "+ str(filtered_df.count()))






24/12/16 15:08:43 WARN MemoryStore: Not enough space to cache rdd_3_7 in memory! (computed 14.7 MiB so far)
24/12/16 15:08:43 WARN MemoryStore: Not enough space to cache rdd_3_6 in memory! (computed 14.7 MiB so far)
24/12/16 15:08:43 WARN MemoryStore: Not enough space to cache rdd_3_8 in memory! (computed 14.9 MiB so far)
24/12/16 15:08:43 WARN MemoryStore: Not enough space to cache rdd_3_9 in memory! (computed 14.7 MiB so far)
24/12/16 15:08:43 WARN MemoryStore: Not enough space to cache rdd_3_10 in memory! (computed 25.4 MiB so far)


Number of rows in victim_df: 29360479


24/12/16 15:08:46 WARN MemoryStore: Not enough space to cache rdd_3_6 in memory! (computed 14.7 MiB so far)
24/12/16 15:08:46 WARN MemoryStore: Not enough space to cache rdd_3_7 in memory! (computed 14.7 MiB so far)
24/12/16 15:08:46 WARN MemoryStore: Not enough space to cache rdd_3_8 in memory! (computed 22.3 MiB so far)
24/12/16 15:08:46 WARN MemoryStore: Not enough space to cache rdd_3_9 in memory! (computed 3.9 MiB so far)
24/12/16 15:08:47 WARN MemoryStore: Not enough space to cache rdd_3_10 in memory! (computed 25.4 MiB so far)


Number of rows in filtered_df: 27573502


In [42]:
# Save the cleaned Victim Records DataFrame as CSV
filtered_df.write.csv("clean_victim_records.csv", header=True, mode="overwrite")
print("Victim Records saved successfully")

24/12/16 15:08:58 WARN MemoryStore: Not enough space to cache rdd_3_7 in memory! (computed 14.7 MiB so far)
24/12/16 15:08:58 WARN MemoryStore: Not enough space to cache rdd_3_6 in memory! (computed 14.7 MiB so far)
24/12/16 15:09:01 WARN MemoryStore: Not enough space to cache rdd_3_8 in memory! (computed 22.3 MiB so far)
24/12/16 15:09:02 WARN MemoryStore: Not enough space to cache rdd_3_9 in memory! (computed 3.9 MiB so far)
24/12/16 15:09:04 WARN MemoryStore: Not enough space to cache rdd_3_10 in memory! (computed 25.4 MiB so far)


Victim Records saved successfully
